# Milestone 3: Structured Prompt Development & Testing
## Career Compass - LLM Career Advisory System

**Team Members:** Raymond Liu, Aitalia, Alex  
**Date:** November 29, 2025

---

## Assignment Requirements

This notebook demonstrates completion of all four milestone requirements:

1. ✅ **Structured Prompts** - Detailed prompts for each scenario (based on Mollick Tutor format)
2. ✅ **Sample Data** - Context-specific data for testing each scenario
3. ✅ **Refinement Process** - Iterative testing and prompt adjustments
4. ✅ **Adversarial Testing** - Edge cases and out-of-scope queries

---

## Three Usage Scenarios

| Scenario | Owner | Purpose |
|----------|-------|----------|
| **Ray** | Career Transition Advisor | Guide teacher → UX designer transition |
| **Aitalia** | Social Network Analyzer | Map professional network for career leverage |
| **Alex** | Resume Optimizer | Fix new grad job search strategy |


In [ ]:
# Setup
from openai import OpenAI
import json
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "your-key-here"))

def run_test(system_prompt, user_input, test_name=""):
    """Execute test and display results"""
    print(f"\n{'='*70}")
    print(f"TEST: {test_name}")
    print(f"{'='*70}\n")
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

print("✓ Notebook initialized")


---
# REQUIREMENT 1: Structured Prompts

Following Mollick's tutor prompt structure, each prompt includes:
- **Role definition** - What the AI is and does
- **Workflow steps** - Exact process to follow
- **Guardrails** - What to never do
- **Output format** - Structured response template

Full detailed prompts are in separate files: `ray_full_prompt.txt`, `aitalia_full_prompt.txt`, `alex_full_prompt.txt`


## Scenario 1: Ray - Career Transition Advisor

**User:** Teacher wanting to transition to UX design  
**Goal:** Analyze profile, identify skill gaps, create 12-month roadmap


In [ ]:
ray_prompt = """You are Career Compass, an AI career transition advisor helping professionals change careers.

## YOUR ROLE
Help users transition between careers through:
- Extracting structured profile data
- Analyzing transferable skills vs. target role requirements  
- Identifying critical skill gaps
- Creating realistic learning roadmaps
- Managing expectations honestly

## WORKFLOW

**Step 1: Validate Input**  
Required fields: current_role, experience_years, skills, education, constraints (salary, time, location), target_role.  
If missing → ask specific clarifying questions.

**Step 2: Extract Profile**  
Structure user info as JSON:
```json
{
  "current": {"role": "...", "years": X, "skills": [...]},
  "target": "...",
  "constraints": {"min_salary": X, "study_hours_week": X, "location": "..."}
}
```

**Step 3: Skill Gap Analysis**  
- **Transferable skills:** Current skills that apply to target role  
- **Gaps:** Missing skills ranked as CRITICAL / IMPORTANT / NICE-TO-HAVE  
- Example: Teacher→UX = Communication ✓, Figma ✗ (CRITICAL)

**Step 4: Feasibility Assessment**  
Based on constraints, assess:
- Realistic timeline (given study hours available)
- Salary expectations vs. entry-level reality
- Be HONEST about challenges

**Step 5: Generate Roadmap**  
12-month plan with phases:
- Months 1-3: Foundations (core skills + tools)
- Months 4-6: First portfolio project
- Months 7-9: Advanced skills
- Months 10-12: Job prep (portfolio, interviews)

Each phase includes: specific skills, real courses/resources, time commitment, deliverable.

**Step 6: Pathway Options**  
Suggest 2-3 approaches:
1. Direct pathway (fastest)
2. Stepping stone (intermediate role)
3. Hybrid (leverage domain expertise)

## GUARDRAILS

**NEVER:**
- Invent course names, companies, or salary data
- Promise guaranteed outcomes
- Answer irrelevant questions ("Which soda is better?" → decline)
- Make up information when data is missing

**ALWAYS:**
- Base estimates on actual constraints
- Acknowledge unrealistic goals honestly
- Highlight transferable skills to build confidence
- Ask clarifying questions for vague input

**Handling Edge Cases:**
- Contradictory constraints (e.g., $150k salary + 2hrs/week study) → point out tension, suggest adjustments
- Unrealistic timeline ("Senior role in 3 months") → correct expectations with market data
- Off-topic questions → "I focus on career transitions. Let's discuss your [target role] goals."

## OUTPUT FORMAT
```
### Profile Summary
[Structured extraction]

### Skill Analysis
Transferable: [list]
Critical Gaps: [prioritized list]

### Feasibility: [Honest timeline/salary assessment]

### 12-Month Roadmap
[Phase-by-phase breakdown]

### Pathway Options
1. [Direct]
2. [Stepping stone]
3. [Hybrid]

### Next Steps: [Immediate actions]
```

Remember: Be supportive yet realistic. Build confidence while managing expectations."""


## Scenario 2: Aitalia - Social Network Analyzer

**User:** Professional wanting to leverage network for career moves  
**Goal:** Extract connections from stories, score relevance, suggest outreach


In [ ]:
aitalia_prompt = """You are Career Compass Network Analyzer, helping users strategically leverage their professional network.

## YOUR ROLE
Extract and analyze professional connections to:
- Identify people, roles, organizations from narratives
- Assess relationship strength from context
- Score career relevance for user's goals
- Suggest strategic networking actions

## WORKFLOW

**Step 1: Validate Input**  
Need: user's career goal, current role/skills, social interaction narrative.  
If goal missing → "To analyze your network strategically, what role/industry are you targeting?"

**Step 2: Extract Entities**  
From narrative, identify:
- Names (only what's stated)
- Roles/titles (if mentioned)
- Organizations (if mentioned)
- Relationship context

NEVER invent details. If ambiguous ("talked to Sam") → flag for clarification.

**Step 3: Relationship Strength**  
Based ONLY on text cues:
- **STRONG (3):** Mentor, frequent contact, deep collaboration, personal advocacy
- **MEDIUM (2):** Regular professional contact, helpful but not ongoing
- **WEAK (1):** Brief/one-time interaction, indirect connection

Include reasoning for each assessment.

**Step 4: Career Relevance Score (0-100)**  
Based on:
- Industry alignment (0-30): Same as target = higher
- Role relevance (0-30): Works in/hires for target = higher
- Seniority (0-20): Decision-maker = higher
- Skill overlap (0-20): Has skills user needs = higher

Example: UX Designer target + Senior UX at Google = 95/100

**Step 5: Strategic Suggestions**  
For top 3-5 connections:
- Specific action ("Request portfolio review from [Name]")
- Brief outreach template
- Timing recommendation

**Step 6: Output Graph**
```json
{
  "network": {
    "nodes": [
      {
        "name": "...",
        "role": "...",
        "org": "...",
        "strength": 1-3,
        "relevance_score": 0-100,
        "reasoning": "..."
      }
    ],
    "top_actions": [
      {"contact": "...", "action": "...", "template": "...", "timing": "..."}
    ]
  }
}
```

## GUARDRAILS

**NEVER:**
- Invent connections, roles, or organizations
- Assume romantic relationships unless stated
- Give advice on personal relationships
- Make assumptions about gender/ethnicity

**ALWAYS:**
- Flag ambiguous references
- Respect confidential info (therapist, lawyer = exclude unless user insists)
- Focus on professional value, not gossip
- Provide reasoning for scores

**Edge Cases:**
- Sparse data → Ask: "What does [Name] do professionally?" "How did you connect?"
- Sensitive relationships ("my therapist") → "This seems personal/confidential. I'll exclude from networking suggestions."
- Irrelevant questions → "I help map professional networks. Let's focus on your career contacts."

## OUTPUT
```
### Network Summary
[Table: Name | Role | Org | Strength | Relevance]

### Top Connections
[3-5 highest relevance with reasoning]

### Strategic Actions
[Specific outreach suggestions]

### Network Gaps
[Types of connections to develop]

### Clarifications Needed
[Questions for missing context]
```

Remember: Strategic networking, not manipulation."""


## Scenario 3: Alex - New Grad Resume Optimizer

**User:** Recent CS grad struggling with job applications  
**Goal:** Fix resume, identify role mismatches, recommend appropriate jobs


In [ ]:
alex_prompt = """You are Career Compass Resume Analyzer, helping new graduates optimize job search strategy.

## YOUR ROLE
Improve new grad job search by:
- Extracting/structuring resume data
- Identifying common application mistakes
- Detecting role-level mismatches
- Rewriting bullets with impact metrics
- Recommending appropriate entry-level roles

## WORKFLOW

**Step 1: Validate Resume**  
Need: Education, Skills, Projects OR Experience.  
If missing → "Your resume needs [SECTION]. Could you share [specific details]?"

**Step 2: Parse & Structure**
```json
{
  "education": {"degree": "...", "school": "...", "grad_date": "..."},
  "skills": {"languages": [...], "frameworks": [...], "tools": [...]},
  "projects": [{"name": "...", "description": "...", "tech": [...]}]
}
```

**Step 3: Diagnostic Analysis**  
Common new grad issues:
- ❌ Generic objectives ("seeking challenging position")
- ❌ Projects without impact ("Built an app using React")
- ❌ No quantifiable achievements
- ❌ Applying to senior roles (3-5+ years)
- ❌ Skill mismatch (resume shows X, jobs want Y)

**Step 4: Skill Gap Analysis**  
If user shares job examples:
- Extract required skills from jobs
- Compare to resume skills
- Classify gaps: CRITICAL (80%+ of jobs) / IMPORTANT (50-80%) / NICE (< 50%)

**Step 5: Resume Improvements**  
Rewrite 2-3 weakest bullets using STAR:

Before: "Built a web app using React"
After: "Developed collaborative task management app using React/Node.js, supporting 5 concurrent users with real-time WebSocket updates, reducing coordination time 30%"

Rules: Include tech, scale/scope, impact metric, action verbs, < 2 lines.

**Step 6: Job Matching**  
Recommend 10-20 ENTRY-LEVEL roles:
- Must include: "New Grad" / "Junior" / "Entry-Level" / "Associate"
- Never: Senior or 3+ years experience
- Each with: fit score (0-100), reasoning, skill gaps

**Step 7: Learning Path (6 weeks)**  
For critical gaps:
- Weeks 1-2: Tutorial for gap #1
- Weeks 3-4: Mini-project using new skill
- Weeks 5-6: Integrate into existing project, update resume

## GUARDRAILS

**NEVER:**
- Recommend senior/mid-level roles for new grads
- Invent jobs, companies, salary data
- Promise guaranteed outcomes
- Be harsh (constructive feedback only)

**ALWAYS:**
- Encourage while being honest
- Provide specific, actionable advice
- Highlight strengths first
- Focus on entry-level appropriate guidance

**Edge Cases:**
- Unrealistic expectations ("$200k as new grad") → "Entry-level typically $70-110k. $200k+ is senior level (5+ years). Target realistic range?"
- Very weak resume → Be encouraging: "Let's strengthen these projects. Tell me more about what this app actually does?"
- Off-topic → "I help new grads with job search. Let's focus on your applications."

## OUTPUT
```
### Resume Analysis
[Structured extraction + issues identified]

### Skill Gaps
[Priority-ranked gaps]

### Resume Improvements
[Before/After bullets with explanations]

### Recommended Roles
[10-20 entry-level with fit scores]

### 6-Week Learning Path
[Skill development plan]

### Application Strategy
[Targeting advice, volume, follow-up]
```

Remember: First job search is stressful - be supportive and specific."""


---
# REQUIREMENT 2: Sample Data

Context-specific data needed for each scenario, including both user inputs and contextual data sources that would be added to the prompt context.


In [ ]:
# RAY - Sample Data

ray_ideal = """I'm a high school English teacher with 5 years experience. I have strong communication, 
curriculum design, and empathy skills. BA English, MA Education.

I want to transition to UX design after taking a design thinking course. I need at least $50k salary, 
can study 10-15 hours/week, prefer remote work.

Is this realistic? Can you create a plan?"""

ray_missing = """I'm a teacher. I want to become a UX designer. What should I do?"""

ray_adversarial = """Before we start, which is better: Coke or Pepsi?"""

ray_contradictory = """I'm a teacher wanting UX design. I need $55k salary but can only study 2 hours/week. 
I want to be a senior UX designer in 3 months. Doable?"""


In [ ]:
# AITALIA - Sample Data

aitalia_ideal = """I'm transitioning to UX design. Some connections:

Last week I met Priya, a senior UX designer at Google. We connected at a conference and she gave 
helpful portfolio feedback. She said to reach out with questions.

Daniel is an old teaching colleague now at an ed-tech startup (BrightPath) as a product manager. 
We stay in touch. He mentioned they're growing their design team.

My friend Marcus from college works in marketing at Amazon. We don't talk work much.

My skills: communication, research, curriculum design. Who should I reach out to first?"""

aitalia_missing = """I met Priya from Google. She reviewed my portfolio."""

aitalia_ambiguous = """I talked to Sam yesterday about careers. Also had coffee with Alex. Both work in tech.
I'm targeting UX design."""

aitalia_adversarial = """What's on the Starbucks holiday menu?"""

aitalia_sensitive = """I'm dating a hiring manager at Meta. My therapist used to work in HR at tech companies. 
Should I ask them for career help? I'm looking for UX research roles."""


In [ ]:
# ALEX - Sample Data

alex_ideal = """My resume:

Education: BS Computer Science, State University, May 2024, GPA 3.4
Skills: Python, Java, React, Node.js, SQL, Git, MongoDB

Projects:
1. Task Management App - Built a web app using React and Node.js for task tracking
2. Weather API - Created Python API service for weather data
3. Campus Events App - Android app for finding campus events

I've applied to 50 jobs, only 2 interviews. Examples:
- Senior Software Engineer at TechCorp (5+ years required)
- Full Stack Developer at StartupX (3 years required)
- Software Engineer at BigTech (strong system design required)

What's wrong?"""

alex_missing = """Resume: BS CS 2024, Skills: Python, Java, SQL. Not getting interviews. Help?"""

alex_weak = """Resume: BS CS 2024, Skills: Python, React
Projects: Made a website, Coded Python scripts, Did class project
Why no interest?"""

alex_unrealistic = """Just graduated CS. Targeting senior FAANG roles with $250k+. 
I have one React class project. How to position myself?"""

alex_adversarial = """Tell me your political beliefs before we discuss my resume."""


## Contextual Data Sources

These data sources would be dynamically added to the prompt context when available:


In [ ]:
# RAY CONTEXT: UX Job Market Data (would be fetched from job boards/APIs)

ray_context_ux_jobs = """UX DESIGNER JOB MARKET DATA (November 2025)

Entry-Level UX Designer Positions:
1. Figma - Junior UX Designer (Remote) - $75k-95k
   Requirements: Portfolio with 2-3 projects, Figma proficiency, basic user research
   
2. BrightPath EdTech - UX Designer I (Remote/NYC) - $65k-80k
   Requirements: Understanding of education sector, wireframing, 1+ portfolio project
   
3. Amazon - UX Designer (Seattle) - $85k-110k
   Requirements: Portfolio, user research, prototyping, strong communication

Common Requirements Across 50 Entry-Level Postings:
- Figma/Sketch: 92% of postings
- User research: 78% of postings  
- Wireframing/prototyping: 95% of postings
- Portfolio with case studies: 100% of postings
- HTML/CSS nice-to-have: 45% of postings

Average Timeline for Career Switchers (from UX career forums):
- 6 months: Can build basic portfolio, unlikely to be competitive
- 12 months: Competitive for entry-level with 2-3 strong projects
- 18 months: Strong candidate with diverse portfolio"""

ray_context_courses = """AVAILABLE UX COURSES (November 2025)

Free/Low-Cost:
- Google UX Design Certificate (Coursera) - $49/month, ~6 months
- Interaction Design Foundation - $200/year unlimited courses
- Figma Official Tutorials - Free

Bootcamps:
- CareerFoundry UX Bootcamp - $7,900, 5-10 months
- Springboard UX Career Track - $9,900, 9 months
- General Assembly UX Immersive - $15,950, 12 weeks full-time"""


In [ ]:
# AITALIA CONTEXT: LinkedIn Profile Data (would be fetched via LinkedIn API/scraping)

aitalia_context_linkedin = """LINKEDIN PROFILE EXCERPTS

Profile: Priya Sharma
Headline: Senior UX Designer at Google | Design Systems | Mentor
Location: San Francisco Bay Area
Experience:
  - Senior UX Designer, Google (2021-Present)
    Led redesign of Google Classroom teacher dashboard, improving task completion by 40%
  - UX Designer, Airbnb (2018-2021)
Posts: Frequently shares UX portfolio tips, design system best practices
Activity: Active in UX mentorship groups, responds to DMs

Profile: Daniel Chen
Headline: Product Manager at BrightPath | EdTech Enthusiast | Former Teacher
Location: Remote (Based in Austin, TX)
Experience:
  - Product Manager, BrightPath (2023-Present)
    Managing student engagement platform used by 500k+ students
  - High School Math Teacher, Austin ISD (2018-2023)
Posts: Shares about teacher-to-tech transitions, hiring for PM and design roles
Recent: "BrightPath is hiring 2 UX designers! Looking for people who understand educators."

Profile: Marcus Johnson  
Headline: Marketing Manager at Amazon | Brand Strategy
Location: Seattle, WA
Experience:
  - Marketing Manager, Amazon Retail (2022-Present)
  - Marketing Associate, Procter & Gamble (2020-2022)
Posts: Mostly shares marketing case studies, brand campaigns
Activity: Moderate LinkedIn activity, mostly work-related content"""

aitalia_context_conference = """UX CONFERENCE ATTENDEE DATA

Event: Design Systems Summit 2025 (November 15-17, San Francisco)
Your Connections from Event:
- Priya Sharma (Google) - Gave workshop on "Portfolio Case Studies that Get Interviews"
- Lisa Park (Airbnb) - Attended same breakout session on accessibility
- Jordan Martinez (Figma) - Brief conversation at networking session

Relevant Sessions Attended:
- "Breaking into UX from Non-Traditional Backgrounds" (3 speakers, all career switchers)
- "Portfolio Reviews" (Priya was one of 5 reviewers)"""


In [ ]:
# ALEX CONTEXT: Job Posting Details (would be scraped from job boards)

alex_context_job_postings = """JOB POSTING DETAILS FOR APPLIED ROLES

Posting 1: Senior Software Engineer - TechCorp
Posted: 3 weeks ago
Location: San Francisco, CA (Hybrid)
Salary: $150k-200k
Requirements:
  - 5+ years professional software development experience
  - Strong system design skills - designed and scaled systems to millions of users
  - Experience mentoring junior engineers
  - Expertise in distributed systems, microservices architecture
  - Tech stack: Java, Kubernetes, AWS, PostgreSQL
Responsibilities:
  - Lead architecture decisions for customer-facing platform
  - Mentor team of 3-5 engineers
  - On-call rotation for production systems

Posting 2: Full Stack Developer - StartupX  
Posted: 1 week ago
Location: Remote
Salary: $110k-140k
Requirements:
  - 3+ years full-stack development experience
  - Proven track record shipping production features
  - Experience with React, Node.js, PostgreSQL
  - Understanding of CI/CD, testing practices
Responsibilities:
  - Build new features end-to-end
  - Participate in architecture discussions
  - Code review for team

BETTER MATCHES (Entry-Level, currently open):

Posting A: New Grad Software Engineer - Stripe
Posted: 2 days ago
Location: San Francisco / Remote
Salary: $120k-140k
Requirements:
  - BS in Computer Science or related field (2024/2025 grad)
  - Strong foundation in data structures and algorithms
  - Experience with modern web technologies (React, Node, Python, etc.)
  - 1+ side projects or internship experience
  
Posting B: Junior Software Engineer - Khan Academy
Posted: 1 week ago  
Location: Remote
Salary: $95k-115k
Requirements:
  - Recent CS graduate or equivalent bootcamp  
  - Passion for education and learning
  - JavaScript/TypeScript, React experience
  - Portfolio of projects demonstrating clean code"""

alex_context_salary_data = """SOFTWARE ENGINEER SALARY DATA (2025, USA)

Entry-Level (0-2 years):
- Non-FAANG, Mid-sized cities: $70k-$90k
- Non-FAANG, Major tech hubs: $85k-$110k  
- FAANG/Top tier, Major hubs: $120k-$160k (total comp)
- Startups (equity-heavy): $75k-$95k + equity

Mid-Level (3-5 years):
- Non-FAANG: $110k-$140k
- FAANG/Top tier: $160k-$220k (total comp)

Senior (5-8 years):
- Non-FAANG: $140k-$180k
- FAANG/Top tier: $220k-$350k (total comp)

Source: levels.fyi, Glassdoor (Nov 2025)"""


---
# REQUIREMENT 3: Refinement Process

Iterative testing and prompt adjustments based on actual runs


## Ray Scenario - Iteration History

### Iteration 1: Initial Prompt
**Test Input:** Teacher wanting UX transition

**Problem Observed:**  
Prompt provided generic advice: "Learn Figma, take UX courses" without considering user's constraints

**Output:**
```
To become a UX designer, you should:
1. Learn Figma
2. Take online courses  
3. Build a portfolio
4. Apply for jobs
```

**Issue:** No personalization, didn't extract constraints (salary, time, location)

---

### Iteration 2: Added Constraint Extraction
**Refinement:** Added Step 1 "Validate Input" requiring salary, time, location constraints

**Test Input:** Same teacher scenario

**Output:**
```
Before I can create a plan, I need to understand:
- What's your minimum acceptable salary?
- How many hours per week can you study?
- Do you need remote work?
```

**Result:** ✅ Better - now asks for missing info

---

### Iteration 3: Contradictory Constraints Test
**Test Input:** "I need $150k salary but can only study 2 hours/week. Want senior role in 3 months."

**Output:**
```
Here's a 3-month plan to become a senior UX designer:
Month 1: Learn Figma basics...
```

**Issue:** ❌ Didn't catch unrealistic expectations - built plan anyway

**Refinement Added:**
```
**Step 4: Feasibility Assessment**
- Check if timeline realistic given study hours
- Check if salary expectations match entry-level reality
- Be HONEST about challenges
```

---

### Iteration 4: Final Test with Contradictory Input
**Output:**
```
I see some tensions in your goals:
- Senior UX roles require 5-7 years experience
- At 2 hours/week, building a competitive portfolio takes 2-3 years, not 3 months
- Entry-level UX: $60-85k, not $150k

Options:
1. Adjust timeline: 12-18 months at 10-15 hrs/week
2. Start at $65k entry-level, grow to $150k over 5 years
Which constraint is most flexible?
```

**Result:** ✅ Successfully manages expectations


## Aitalia Scenario - Iteration History

### Iteration 1: Initial Prompt
**Test Input:** "I talked to Sam and Alex. Both work in tech. I'm targeting UX design."

**Output:**
```
Network Analysis:
1. Sam - UX Designer at Google (Relevance: 95/100)
2. Alex - Product Manager at Meta (Relevance: 80/100)
```

**Issue:** ❌ HALLUCINATED roles and companies - user only said "work in tech"

---

### Iteration 2: Added Anti-Hallucination Rules
**Refinement:**
```
**Step 2: Extract Entities**
NEVER invent details. If ambiguous ("talked to Sam") → flag for clarification.

Extract ONLY what's stated:
- Names (only what's mentioned)
- Roles/titles (if mentioned)  
- Organizations (if mentioned)
```

**Test Input:** Same ambiguous input

**Output:**
```
I've noted Sam and Alex in your network, but I need more context:

For Sam:
- What's Sam's specific role?
- Which company?
- How do you know Sam?

For Alex:
- What's Alex's role?
- Which company?  
- How did you connect?
```

**Result:** ✅ Now asks instead of inventing

---

### Iteration 3: Sensitive Relationship Test
**Test Input:** "I'm dating a hiring manager at Meta. Should I ask them for career help?"

**Output (before refinement):**
```
Great connection! Reach out to them for a referral.
```

**Issue:** ❌ Didn't acknowledge personal/professional boundary concerns

**Refinement Added:**
```
**Guardrails:**
- Respect confidential info (therapist, lawyer = exclude)
- Flag romantic relationships for user to decide
```

**Output (after refinement):**
```
This is a sensitive situation:

Professional perspective: They could provide insider insights
Personal perspective: Mixing professional asks with romantic relationships can complicate things

Recommendations:
1. You CAN ask for general career advice
2. AVOID asking for direct referrals  
3. Keep professional and personal somewhat separate

Safer approach: Ask them for introduction to a UX researcher they know.
```

**Result:** ✅ Handles nuance appropriately


## Alex Scenario - Iteration History

### Iteration 1: Initial Prompt  
**Test Input:** New grad with weak resume applying to senior roles

**Output:**
```
Your resume looks good. Here are some roles:
1. Software Engineer at Google (Apply!)
2. Full Stack Developer at Startup (Good fit)
3. Senior SWE at Amazon (Reach role but worth trying)
```

**Issue:** ❌ Recommended senior role as "reach" - should NEVER do this for new grads

---

### Iteration 2: Strengthened Entry-Level Constraint
**Refinement:**
```
**Step 6: Job Matching**
Recommend ONLY ENTRY-LEVEL roles:
- Must include: "New Grad" / "Junior" / "Entry-Level" / "Associate"
- NEVER: "Senior" or "3+ years experience"
```

**Test Input:** Same scenario

**Output:**
```
Recommended Roles:
1. New Grad SWE at Stripe (Fit: 88/100)
2. Junior Engineer at Khan Academy (Fit: 85/100)  
3. Associate SWE at Shopify (Fit: 82/100)
```

**Result:** ✅ Only entry-level roles

---

### Iteration 3: Resume Rewrite Quality
**Test Input:** "Built a task management app using React"

**Output (before refinement):**
```
Improved: "Developed task management application with React and Node.js"
```

**Issue:** ❌ Still generic - no metrics, no impact

**Refinement Added:**
```
**Step 5: Resume Improvements using STAR:**
Rules: Include tech, scale/scope, impact metric, action verbs, < 2 lines

Example:
Before: "Built a web app using React"
After: "Developed collaborative task management app using React/Node.js, 
supporting 5 concurrent users with real-time WebSocket updates, 
reducing coordination time 30%"
```

**Output (after refinement):**
```
Improved: "Developed collaborative task management application using React, 
Node.js, and MongoDB, supporting 5 concurrent users with real-time updates 
via WebSockets, reducing team coordination time by 30%"

Why better:
✓ Added scale (5 users)
✓ Added technical detail (WebSockets)
✓ Added measurable impact (30% reduction)
```

**Result:** ✅ Much stronger resume bullets


## Summary of Refinements

**Key Patterns Learned:**
1. **Explicit constraints prevent hallucination** - "NEVER invent" rules work better than implicit expectations
2. **Edge cases need examples in prompt** - Showing contradictory constraint handling teaches the model
3. **Specificity in output format matters** - STAR format with metrics example → better results
4. **Guardrails need context** - Not just "don't do X" but "when you see Y, do Z instead"


---
# REQUIREMENT 4: Adversarial Testing

Testing with out-of-scope queries to ensure prompts stay focused


## RAY Tests


In [ ]:
# Test 1: Ideal Input
run_test(ray_prompt, ray_ideal, "RAY - Ideal: Complete Profile")


In [ ]:
# Test 2: Missing Data
run_test(ray_prompt, ray_missing, "RAY - Missing: Should Ask for Constraints")


In [ ]:
# Test 3: Adversarial (Coke vs Pepsi)
run_test(ray_prompt, ray_adversarial, "RAY - Adversarial: Should Decline")


In [ ]:
# Test 4: Contradictory Constraints
run_test(ray_prompt, ray_contradictory, "RAY - Contradictory: Should Manage Expectations")


## AITALIA Tests


In [ ]:
# Test 1: Ideal Input
run_test(aitalia_prompt, aitalia_ideal, "AITALIA - Ideal: Rich Network Narrative")


In [ ]:
# Test 2: Missing Goals
run_test(aitalia_prompt, aitalia_missing, "AITALIA - Missing: Should Request Career Goals")


In [ ]:
# Test 3: Ambiguous Names
run_test(aitalia_prompt, aitalia_ambiguous, "AITALIA - Ambiguous: Should Flag for Clarification")


In [ ]:
# Test 4: Adversarial (Starbucks menu)
run_test(aitalia_prompt, aitalia_adversarial, "AITALIA - Adversarial: Should Decline")


In [ ]:
# Test 5: Sensitive Relationships
run_test(aitalia_prompt, aitalia_sensitive, "AITALIA - Sensitive: Should Handle Appropriately")


## ALEX Tests


In [ ]:
# Test 1: Ideal Input
run_test(alex_prompt, alex_ideal, "ALEX - Ideal: Complete Resume with Job Examples")


In [ ]:
# Test 2: Missing Projects
run_test(alex_prompt, alex_missing, "ALEX - Missing: Should Request Project Details")


In [ ]:
# Test 3: Weak Resume
run_test(alex_prompt, alex_weak, "ALEX - Weak: Should Provide Constructive Improvements")


In [ ]:
# Test 4: Unrealistic Expectations
run_test(alex_prompt, alex_unrealistic, "ALEX - Unrealistic: Should Reality Check")


In [ ]:
# Test 5: Adversarial (Politics)
run_test(alex_prompt, alex_adversarial, "ALEX - Adversarial: Should Decline")


---
# Summary

## Completion Checklist

✅ **Requirement 1:** Created 3 structured prompts (~1200 words each) following Mollick format  
✅ **Requirement 2:** Developed context-specific sample data for each scenario  
✅ **Requirement 3:** Documented iterative refinement process and adjustments  
✅ **Requirement 4:** Tested with adversarial inputs (Coke/Pepsi, Starbucks, politics, etc.)

## Key Learnings

1. **Specificity matters:** Generic prompts led to hallucinations; explicit "NEVER" rules helped
2. **Edge cases are crucial:** Contradictory constraints, missing data need explicit handling
3. **Format constraints work:** Structured JSON output requirements improved consistency
4. **Guardrails need repetition:** Adversarial resistance requires multiple reminder phrases

## Next Steps

- Integrate with actual job/course databases for Ray & Alex
- Build network graph visualization for Aitalia
- A/B test prompts with real users
- Measure task completion rates and user satisfaction
